<a href="https://colab.research.google.com/github/JoseAlbertoSeco/BlablacarCarpooling_DataMining/blob/Patterns/notebooks/Clasificaci%C3%B3n_multiobjetivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Línea de trabajo 3:

En este notebook se realizará una clasificación multiobjetivo, con ello se pretende que la empresea prevea la ruta que se va a hacer.

Con esta predicción se pretende ayudar a el usuario para ver qué rutas son probables que se realicen

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn import linear_model
from sklearn import tree 

from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.multiclass import unique_labels

# Carga de datos

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
clm = pd.read_csv("/content/drive/MyDrive/Datos/df_castilla.csv") 
clm.head(2)

,Unnamed: 0,DIA,AÑO,SEMANA,MES,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,Festivo
0,453525,2017-11-05,2017,44,11,es,Albacete,Guadalajara,11.0,1,6,1,6,1,Castilla-La Mancha,Castilla-La Mancha,Albacete,Guadalajara,No,No
1,453526,2017-11-12,2017,45,11,es,Albacete,Guadalajara,11.0,1,6,1,6,1,Castilla-La Mancha,Castilla-La Mancha,Albacete,Guadalajara,No,No


In [4]:
andalucia = pd.read_csv("/content/drive/MyDrive/Datos/df_andalucia.csv") 
andalucia.head(2)

,Unnamed: 0,DIA,AÑO,SEMANA,MES,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,Festivo
0,285649,2017-11-01,2017,44,11,es,Albox,Granada,1.0,1,1,1,1,0,Andalucía,Andalucía,Almería,Granada,No,Si
1,285650,2017-11-03,2017,44,11,es,Albox,Granada,12.0,1,4,1,4,0,Andalucía,Andalucía,Almería,Granada,No,No


In [5]:
clm = clm.drop(clm.columns[[0]], axis=1)
andalucia = andalucia.drop(andalucia.columns[[0]], axis=1)

# Label Encoding

En este apartado transformaremos las variables categóricas (no numéricas) que nos sean necesarias en variables numéricas.

Por ejemplo, en este caso la columna de origen, destino e interprovincial pasarán a ser numéricas {Origen: 'Almansa':41, Destino: 'Puertollano':87, Interprovincial: 'Si':1}.

## Castilla-la Mancha

In [6]:
clm_encoded = clm.copy()

In [7]:
clm_encoded['PROVINCIA_ORIGEN_E'] = clm['PROVINCIA_ORIGEN'].astype('category').cat.codes
clm_encoded['PROVINCIA_DESTINO_E'] = clm['PROVINCIA_DESTINO'].astype('category').cat.codes
clm_encoded['INTERPROVINCIAL_E'] = clm['INTERPROVINCIAL'].astype('category').cat.codes

In [8]:
clm_encoded.tail(2)

,DIA,AÑO,SEMANA,MES,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,Festivo,PROVINCIA_ORIGEN_E,PROVINCIA_DESTINO_E,INTERPROVINCIAL_E
20860,2018-07-14,2018,28,7,es,Guadalajara,Valdelcubo,3.0,1,1,1,1,0,Castilla-La Mancha,Castilla-La Mancha,Guadalajara,Guadalajara,Si,No,4,4,1
20861,2018-08-10,2018,32,8,es,Guadalajara,Cobeta,5.0,1,2,1,1,0,Castilla-La Mancha,Castilla-La Mancha,Guadalajara,Guadalajara,Si,No,4,4,1


In [9]:
clm_encoded = clm_encoded.drop(labels=['CA_ORIGEN', 'CA_DESTINO', 'PROVINCIA_ORIGEN', 'PROVINCIA_DESTINO', 'INTERPROVINCIAL', 'PAIS'], axis=1)
clm_encoded.head(2)

,DIA,AÑO,SEMANA,MES,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,Festivo,PROVINCIA_ORIGEN_E,PROVINCIA_DESTINO_E,INTERPROVINCIAL_E
0,2017-11-05,2017,44,11,Albacete,Guadalajara,11.0,1,6,1,6,1,No,0,4,0
1,2017-11-12,2017,45,11,Albacete,Guadalajara,11.0,1,6,1,6,1,No,0,4,0


In [10]:
clm_encoded = clm_encoded.rename(columns={'CA_ORIGEN_E':'CA_ORIGEN', 'CA_DESTINO_E': 'CA_DESTINO','PROVINCIA_ORIGEN_E': 'PROVINCIA_ORIGEN','PROVINCIA_DESTINO_E': 'PROVINCIA_DESTINO','INTERPROVINCIAL_E': 'INTERPROVINCIAL'})

In [11]:
clm_encoded.head(2)

,DIA,AÑO,SEMANA,MES,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,Festivo,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL
0,2017-11-05,2017,44,11,Albacete,Guadalajara,11.0,1,6,1,6,1,No,0,4,0
1,2017-11-12,2017,45,11,Albacete,Guadalajara,11.0,1,6,1,6,1,No,0,4,0


## Andalucía

In [12]:
andalucia_encoded = andalucia.copy()

In [13]:
andalucia_encoded['PROVINCIA_ORIGEN_E'] = andalucia['PROVINCIA_ORIGEN'].astype('category').cat.codes
andalucia_encoded['PROVINCIA_DESTINO_E'] = andalucia['PROVINCIA_DESTINO'].astype('category').cat.codes
andalucia_encoded['INTERPROVINCIAL_E'] = andalucia['INTERPROVINCIAL'].astype('category').cat.codes

In [14]:
andalucia_encoded.tail(2)

,DIA,AÑO,SEMANA,MES,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL,Festivo,PROVINCIA_ORIGEN_E,PROVINCIA_DESTINO_E,INTERPROVINCIAL_E
246053,2018-06-08,2018,23,6,es,Mairena del Aljarafe,Corteconcepción,3.0,2,1,1,1,1,Andalucía,Andalucía,Sevilla,Huelva,No,No,8,4,0
246054,2018-07-06,2018,27,7,es,Tomares,Algámitas,4.0,1,1,1,1,0,Andalucía,Andalucía,Sevilla,Sevilla,Si,No,8,8,1


In [15]:
andalucia_encoded = andalucia_encoded.drop(labels=['CA_ORIGEN', 'CA_DESTINO', 'PROVINCIA_ORIGEN', 'PROVINCIA_DESTINO', 'INTERPROVINCIAL', 'PAIS'], axis=1)

In [16]:
andalucia_encoded = andalucia_encoded.rename(columns={'PROVINCIA_ORIGEN_E': 'PROVINCIA_ORIGEN','PROVINCIA_DESTINO_E': 'PROVINCIA_DESTINO','INTERPROVINCIAL_E': 'INTERPROVINCIAL'})

In [17]:
andalucia_encoded.head(2)

,DIA,AÑO,SEMANA,MES,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,Festivo,PROVINCIA_ORIGEN,PROVINCIA_DESTINO,INTERPROVINCIAL
0,2017-11-01,2017,44,11,Albox,Granada,1.0,1,1,1,1,0,Si,0,3,0
1,2017-11-03,2017,44,11,Albox,Granada,12.0,1,4,1,4,0,No,0,3,0


# Clasificacion multioutput

Esta estrategia consiste en ajustar un clasificador por cada objetivo. Esto permite clasificar múltiples variables objetivo. 

Para ello utilizaremos la clase Multioutput de sklearn. El propósito de esta clase es ampliar los estimadores para poder estimar una serie de funciones objetivo (f1,f2,f3...,fn) que se entrenan en una única matriz de predicción X para predecir una serie de respuestas (y1,y2,y3...,yn). Se trata de una estrategia sencilla para ampliar los clasificadores que no admiten de forma nativa la clasificación multiobjetivo.


In [18]:
features_selected = ['MES','AÑO','INTERPROVINCIAL','ASIENTOS_OFERTADOS','VIAJES_OFERTADOS']

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
import numpy as np


X = clm_encoded[features_selected]
y = clm_encoded[['DESTINO','ORIGEN','Festivo']]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

y_train_origen = (list((y_train['ORIGEN'].tolist())))
y_train_destino = (list((y_train['DESTINO'].tolist())))
y_train_festivo = (list((y_train['Festivo'].tolist())))

y_multilabel = np.c_[y_train_origen, y_train_destino,y_train_festivo]

classifier_clm = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=2, n_jobs=-1))
classifier_clm.fit(X_train, y_multilabel)

predictions = classifier_clm.predict(X_test)

cross_val_predict(classifier_clm, X_train, y_train, cv=3)

array([['Albacete', 'Ciudad Real', 'No'],
       ['Camuñas', 'Molina de Aragón', 'No'],
       ['Albacete', 'Guadalajara', 'No'],
       ...,
       ['Cuenca', 'Albacete', 'No'],
       ['Albacete', 'Albacete', 'No'],
       ['Ciudad Real', 'Albacete', 'No']], dtype=object)

In [20]:
classifier_clm.score(X_train, y_train)

0.006368554406628775

In [21]:
X = andalucia_encoded[features_selected]
y = andalucia_encoded[['DESTINO','ORIGEN','Festivo']]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

y_train_origen = (list((y_train['ORIGEN'].tolist())))
y_train_destino = (list((y_train['DESTINO'].tolist())))
y_train_festivo = (list((y_train['Festivo'].tolist())))

y_multilabel = np.c_[y_train_origen, y_train_destino,y_train_festivo]

classifier_andalucia = MultiOutputClassifier(KNeighborsClassifier(n_jobs=-1))
classifier_andalucia.fit(X_train, y_multilabel)

predictions = classifier_andalucia.predict(X_test)

cross_val_predict(classifier_andalucia, X_train, y_train, cv=3)

array([['Granada', 'Almería', 'No'],
       ['Almería', 'Almería', 'No'],
       ['Almería', 'Algeciras', 'No'],
       ...,
       ['Sevilla', 'Armilla', 'No'],
       ['Córdoba', 'Albuñol', 'No'],
       ['Almería', 'Marbella', 'Si']], dtype=object)

In [22]:
classifier_andalucia.score(X_test, y_test)

0.008697183575599116

Modelo bastante bueno.

# Predicciones

Ahora es el turno de predecir. Primero, el usuario deberá facilitar al sistema ciertos valores como: mes, año, si es interprovincial o no, el número de asientos y el número de viajes.

## Predicción para el usuario en Castilla-la Mancha

In [23]:
from collections import Counter

In [24]:
interprovincial_clm = clm.INTERPROVINCIAL.astype('category')
interprovincial_dic_clm =dict(enumerate(interprovincial_clm.cat.categories))

In [25]:
#@title Mes del viaje

mes_clm = 3 #@param {type:"slider", min:1, max:12, step:1}

In [26]:
#@title Año del viaje

año_clm = 2022 #@param [2021, 2022, 2023, 2024] {type:"raw"}

In [27]:
#@title Interprovincial

interprovincial_clm = True #@param [True, False] {type:"raw"}
interprovincial_clm = 0 if interprovincial_clm == False else 1
interprovincial_clm

1

In [28]:
#@title Número de asientos

asientos_clm = 4 #@param {type:"slider", min:1, max:8, step:1}

In [29]:
#@title Número de viajes que harías

viajes_clm = 5 #@param {type:"slider", min:1, max:10, step:1}

In [30]:
dic_caracteristicas_clm = {'MES':mes_clm,'AÑO':año_clm,'INTERPROVINCIAL':interprovincial_clm,'ASIENTOS_OFERTADOS':asientos_clm,  'VIAJES_OFERTADOS':viajes_clm}
df_viaje_clm = pd.DataFrame(dic_caracteristicas_clm, index=[0])
df_viaje_clm

,MES,AÑO,INTERPROVINCIAL,ASIENTOS_OFERTADOS,VIAJES_OFERTADOS
0,3,2022,1,4,5


### Resultado

In [31]:
pred_clm =  classifier_clm.predict(df_viaje_clm)

print("Ruta que se realizará\n Origen: {origen}\n Destino: {destino}\n Festivo: {festivo}".format(origen = pred_clm[0][0],destino=  pred_clm[0][1], festivo = pred_clm[0][2]))

Ruta que se realizará
 Origen: Albacete
 Destino: Alcázar de San Juan
 Festivo: No


## Predicción para el usuario en Andalucía

In [32]:
interprovincial_andalucia = andalucia.INTERPROVINCIAL.astype('category')
interprovincial_dic_andalucia =dict(enumerate(interprovincial_andalucia.cat.categories))

In [33]:
#@title Mes del viaje

mes_andalucia = 3 #@param {type:"slider", min:1, max:12, step:1}

In [34]:
#@title Año del viaje

año_andalucia = 2022 #@param [2021, 2022, 2023, 2024] {type:"raw"}

In [35]:
#@title Interprovincial

interprovincial_andalucia = False #@param ["False", "True"] {type:"raw"}
interprovincial_andalucia = 0 if interprovincial_andalucia == False else 1
interprovincial_andalucia

0

In [36]:
#@title Número de asientos

asientos_andalucia = 4 #@param {type:"slider", min:1, max:8, step:1}

In [37]:
#@title Número de viajes que harías

viajes_andalucia = 5 #@param {type:"slider", min:1, max:10, step:1}

In [38]:
dic_caracteristicas_andalucia = {'MES':mes_andalucia,'AÑO':año_andalucia,'INTERPROVINCIAL':interprovincial_andalucia,'ASIENTOS_OFERTADOS':asientos_andalucia,  'VIAJES_OFERTADOS':viajes_andalucia}
df_viaje_andalucia = pd.DataFrame(dic_caracteristicas_andalucia, index=[0])
df_viaje_andalucia

,MES,AÑO,INTERPROVINCIAL,ASIENTOS_OFERTADOS,VIAJES_OFERTADOS
0,3,2022,0,4,5


### Resultado

In [39]:
pred_andalucia =  classifier_andalucia.predict(df_viaje_clm)

print("Ruta que se realizará\n Origen: {origen}\n Destino: {destino}\n Festivo: {festivo}".format(origen = pred_andalucia[0][0],destino= pred_andalucia[0][1], festivo = pred_andalucia[0][2]))

Ruta que se realizará
 Origen: Estepona
 Destino: Granada
 Festivo: No


# Guardar modelos

Nos descargamos los modelos obtenidos.

In [42]:
from joblib import dump, load
dump(classifier_clm, 'clasificador_multiobjetivo_clm.joblib') 

['clasificador_multiobjetivo_clm.joblib']

In [43]:
dump(classifier_andalucia, 'classifier__multiobjetivo_andalucia.joblib') 

['classifier__multiobjetivo_andalucia.joblib']